In [1]:
import sys
print(sys.version)
import numpy as np
from matplotlib import rcParams
from collections import Counter
import re
from matplotlib import colors as mcolors
from os import listdir
from os.path import isfile, join
KT = 3
import Bio.Phylo
from ipynb.fs.full.commonFunctions import *
from ipynb.fs.full.OutPuts import *
from ipynb.fs.full.InPut import *
from ipynb.fs.full.Code import *
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

3.8.8 (default, Apr 13 2021, 12:59:45) 
[Clang 10.0.0 ]


In [2]:
# compilation figure
from ete3 import NCBITaxa, AttrFace, TreeStyle
from ete3 import Tree
import os
def compilationFigure(data,mypath,subcluster):
    
    # open database NCIB
    ncbi = NCBITaxa()
    
    # getting the reference
    
    reference = data['reference']
    referenceChart = [0]*len(reference)
    test = open(mypath+'/CompiledFigure/'+subcluster+"_ref.txt",'w')
    treeFile = open(mypath+'/CompiledFigure/'+subcluster+"_common.txt",'w')
    
    
    # getting list of colours available in Python
    colors = ['blue','green','cyan','magenta','red','yellow','orange','gold','pink','purple']
    
    # getting the 3 most common safety Windows
    safetyIntervals = data['safety windows intervals']
    
    uniqueSafetyIntervals = list()
    [[uniqueSafetyIntervals.append(item) for item in safetyIntervals[i] if not item in uniqueSafetyIntervals] for i in safetyIntervals]
    
    uniqueIntervalsFrequence = [0]*len(uniqueSafetyIntervals)
    
    for i in safetyIntervals:
        for item in safetyIntervals[i]:
            uniqueIntervalsFrequence[uniqueSafetyIntervals.index(item)] += 1
    
    
    tenMostCommonWindows = [window[1] for window in sorted(zip(uniqueIntervalsFrequence, uniqueSafetyIntervals), reverse=True)[:2]]
    tenMostCommonSequences = [window[0] for window in sorted(zip(uniqueIntervalsFrequence, uniqueSafetyIntervals), reverse=True)[:2]]
    
    # printing the reference
    print(" ",file=test)
    print(reference,file=test)
    for window in tenMostCommonWindows:
        print(reference[int(window[0]):int(window[1])],file=treeFile)
    
    
    print(" ",file=test)
    print(" ",file=test)
    test.close()
    treeFile.close()
    
    # creating folders and an array of file
    
    # You should change 'test' to your preferred folder.
    MYDIR = (mypath+"/WindowsFasta")
    CHECK_FOLDER = os.path.isdir(MYDIR)

    # If folder doesn't exist, then create it.
    if not CHECK_FOLDER:
        os.makedirs(MYDIR)
    
    fileWindowsFasta = [mypath+'/WindowsFasta/'+subcluster+"_"+str(i)+".fasta" for i in range(0,len(tenMostCommonWindows)+1)]
    file_fasta_opened = [open(test,'w') for test in fileWindowsFasta]
    
    # getting sequences
    sequences = data['sequences']
    windowOnSequences = data['sequences windows intervals']
    
    taxonomyID = data['taxonomy ID']
    indicatorID = data['indicator ID']
    
    # print all files
    
    for window in tenMostCommonWindows:
        #print("",file=file_opened[tenMostCommonWindows.index(window)])
        idWithinCommonWindow = set()
        for k in range(1,len(sequences)+1):
            if window in safetyIntervals[k]:
                #print(color(''.join(['S',str(k),"\t| "]),fore=colors[uniqueSafetyIntervals.index(window)%10]),end="",file=file_opened[tenMostCommonWindows.index(window)])
                #print(color(''.join([' ' for i in range(0,int(window[0]))]),fore = 'white'),end="",file=file_opened[tenMostCommonWindows.index(window)])
                #print(color(sequences[k][int(windowOnSequences[k][safetyIntervals[k].index(window)][0]):int(windowOnSequences[k][safetyIntervals[k].index(window)][1])+1],fore=colors[uniqueSafetyIntervals.index(window)%10]),end="",file=file_opened[tenMostCommonWindows.index(window)])
                
                # fasta file 
                print(''.join([">",taxonomyID[k],"|",indicatorID[k]]),file=file_fasta_opened[tenMostCommonWindows.index(window)])
                print(''.join([sequences[k][int(windowOnSequences[k][safetyIntervals[k].index(window)][0]):int(windowOnSequences[k][safetyIntervals[k].index(window)][1])+1]]),file=file_fasta_opened[tenMostCommonWindows.index(window)])
                
                #print(color(''.join([' ' for i in range(int(window[1]),len(reference))]),fore = 'white'),end="",file=file_opened[tenMostCommonWindows.index(window)])
                #print(color(" \t| ",fore=colors[uniqueSafetyIntervals.index(window)%10]),end="",file=file_opened[tenMostCommonWindows.index(window)])
                #print(color(''.join([' ',taxonomyID[k]]),fore=colors[uniqueSafetyIntervals.index(window)%10]),end="",file=file_opened[tenMostCommonWindows.index(window)])
                #print("",file=file_opened[tenMostCommonWindows.index(window)])
                
                idWithinCommonWindow.add(taxonomyID[k])
        
        tree = ncbi.get_topology(idWithinCommonWindow)

        def my_layout(node):
            if getattr(node, "rank", None):
                rank_face = AttrFace("rank",fsize=7, fgcolor="indianred")
                node.add_face(rank_face, column=0, position="branch-top")
            if node.is_leaf() == False:
                sciname_face = AttrFace("sci_name", fsize=7, fgcolor="green")
                node.add_face(sciname_face, column=0, position="branch-top")
            if node.is_leaf():
                sciname_face = AttrFace("sci_name", fsize=9, fgcolor="steelblue")
                node.add_face(sciname_face, column=0, position="branch-right")
        ts = TreeStyle()
        ts.layout_fn = my_layout
        ts.show_leaf_name = False
        tree.render(mypath+'/CompiledFigure/'+subcluster+"-"+str(tenMostCommonWindows.index(window))+".png", tree_style=ts,w=500,h=500, units="mm",dpi=100)
        

        
    
    [file.close() for file in file_fasta_opened] 
    
    # running R script for seqlogos and tables
    Pattern = subcluster
    Path = "".join(["/",MYDIR])
    
    commandLine = "".join(["Rscript --vanilla SeqLogo.R ",Path,' ',Pattern])
    os.system(commandLine)
    
    # running R script for references and windows
    Pattern = subcluster
    Path = "/"+mypath + "/CompiledFigure"
    commandLine = "".join(["Rscript --vanilla PrintReference.R ",Path,' ',Pattern])
    os.system(commandLine)
    
    # print taxonomy tree for cluster
    taxonomyID = set(data['taxonomy ID'])
    
    tree = ncbi.get_topology(taxonomyID)
    
    def my_layout(node):
        if getattr(node, "rank", None):
            rank_face = AttrFace("rank",fsize=7, fgcolor="indianred")
            node.add_face(rank_face, column=0, position="branch-top")
        if node.is_leaf() == False:
            sciname_face = AttrFace("sci_name", fsize=7, fgcolor="green")
            node.add_face(sciname_face, column=0, position="branch-top")
        if node.is_leaf():
            sciname_face = AttrFace("sci_name", fsize=9, fgcolor="steelblue")
            node.add_face(sciname_face, column=0, position="branch-right")
    ts = TreeStyle()
    ts.layout_fn = my_layout
    ts.show_leaf_name = False
    tree.render(mypath+'/CompiledFigure/'+subcluster+".png", tree_style=ts,w=500,h=500, units="mm",dpi=100)
    
    
    return 0;

In [3]:
def treeBranchesCalculation(data):
    
    # input files
    sequences = data['sequences']
    tuples = data['tuples']
    taxonomyID = data['taxonomy ID']
    branchesPerTuples = list()
    ncbi = NCBITaxa()
    
    # processing the trees
    for i in range(0,len(tuples)):
        taxonsForTree = set()
        for j in tuples[i]:
            taxonsForTree.add(taxonomyID[j])
        
        tree = ncbi.get_topology(taxonsForTree)
        
        #branchesPerTuples[i] = len(tree)
        for node in tree.traverse("postorder"):
          # Do some analysis on node
          print(node.name)
        
    
    return data

In [4]:
def treeComparison(data):
    
    # input files
    sequences = data['sequences']
    tuples = data['tuples']
    taxonomyID = data['taxonomy ID']
    branchesPerTuples = list()
    ncbi = NCBITaxa()
    
    # processing the trees
    for i in range(0,len(tuples)):
        for j in range(0,len(tuples)):
            
            if i != j: 
                
                # tree i
                taxonsForTree = set()
                for k in tuples[i]:
                    taxonsForTree.add(taxonomyID[k])
                t1 = ncbi.get_topology(taxonsForTree)
                
                # tree j
                taxonsForTree = set()
                for k in tuples[j]:
                    taxonsForTree.add(taxonomyID[k])
                t2 = ncbi.get_topology(taxonsForTree)
                
                # comparision between trees:
                rf, rf_max, common_attrs,edges_t1, edges_t2, discarded_edges_t1, discarded_edges_t2 = t2.robinson_foulds(t1,unrooted_trees=True)
                
                #  
                print(len(common_attrs))
                
    return data;
    

In [5]:
### setting up folder
mypath = 'DataBackup/Backup-HL/Data_20_HL'

# read files and files names
allFiles = [f for f in listdir(mypath+'/safety/') if isfile(join(mypath+'/safety/', f))]
allFilesNames = [f.split('.')[0] for f in allFiles]

output = open("Data_20_HL.txt","a")

# applying algorithms
for k in range(32,33):
    i = allFilesNames[k]
    
    print(i,k)
    # open hmmsearchFile
    hmmsearchFile = mypath+'/hmmsearch/'+i+'.out'
    
    # process scores, e-values and other data from HMMSearch
    data = readscore(hmmsearchFile)

    # print scores, e-values and other data from HMMSearch In Subplots
    #data = scoredisplay(data,mypath,i)
    
    # open safety file
    safetyfile = mypath+'/safety/'+i+'.out'
    
    # update the data with safety windows information
    data = readSafety(data,safetyfile)
    
    # process safety intervals into safety windows
    data= safetyWindows(data)
    
    # solving overlapping windows issue
    #data = overlappingWindowsSize(data)
    
    # print coloured windows
    #data = printColouredWindows(data,mypath,i)
    
    # print colour-code windows
    #data = printCodedWindows(data,mypath,i)
    
    # create tuples
    data = createTuples(data,mypath,i,KT)
    
    # produce cluster  data
    # data = coverageAverage(data)
    
    # display cluster data
    #data = plotResults(data,mypath,i)
    
    # produce tuple average
    # data = TupleAverage(data)
    
    # produce subcliques data 
    #data = subCliques(data)
    
    # display graph data
    #data = plotTupleResults(data,mypath,i)
    
    # produce graph average
    #data = safetyGraph(data)
    
    # tree calculations
    #data = treeBranchesCalculation(data)
    data = treeComparison(data)
    
    
    #save file for compilation figure
    # compilationFigure(data,mypath,i)
    
    #compilation table with all results
    # compilationTable(data,i,output)
    
    #print(hmmsearchFile)
    
output.close()

print("OVAH 20 ")

A7MEI2 32
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
2
2
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
